In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from torch.utils.data import random_split

In [2]:
print(torch.__version__)              # 2.5.1+cu118
print(torch.cuda.is_available())      # True
print(torch.cuda.get_device_name(0))  # Your GPU name

2.5.1+cu118
True
NVIDIA GeForce RTX 3060 Laptop GPU


# Data Prep

In [3]:
file_paths = {
    "TomQuality": "D:/Bitchass Agri stupid af shjt/Reference/TomQuality.csv",
    "GreenhouseClimate": "D:/Bitchass Agri stupid af shjt/Reference/GreenhouseClimate.csv",
    "GrodanSens": "D:/Bitchass Agri stupid af shjt/Reference/GrodanSens.csv",
    "Resources": "D:/Bitchass Agri stupid af shjt/Reference/Resources.csv",
    "Weather": "D:/Bitchass Agri stupid af shjt/Weather/Weather.csv",
    "CropParameters": "D:/Bitchass Agri stupid af shjt/Reference/CropParameters.csv",
    "Production": "D:/Bitchass Agri stupid af shjt/Reference/Production.csv"
}

In [4]:
dfs = {}
for name, path in file_paths.items():
    df = pd.read_csv(path, low_memory=False)
    df.columns = df.columns.str.strip().str.replace('\t', '')
    # === Safe and robust date parsing ===
    if name == 'TomQuality':
        # Use index as the true datetime source
        df.reset_index(inplace=True)
        df['Date'] = pd.to_datetime(df['index'], origin='1899-12-30', unit='D')
    elif '%time' in df.columns:
        df['%time'] = pd.to_numeric(df['%time'], errors='coerce')
        df['Date'] = pd.to_datetime(df['%time'], origin='1899-12-30', unit='D')
    elif 'time' in df.columns:
        df['time'] = pd.to_numeric(df['time'], errors='coerce')
        df['Date'] = pd.to_datetime(df['time'], origin='1899-12-30', unit='D')
    elif 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    else:
        df['Date'] = pd.NaT  # Fallback if no date column
    # Convert all columns except Date to numeric
    df.loc[:, df.columns != 'Date'] = df.loc[:, df.columns != 'Date'].apply(pd.to_numeric, errors='coerce')
    dfs[name] = df

In [5]:
print(dfs['TomQuality'][['index', 'Date']].head())

   index       Date
0  43880 2020-02-19
1  43894 2020-03-04
2  43908 2020-03-18
3  43922 2020-04-01
4  43936 2020-04-15


In [6]:
soil_cols = ['EC_slab1', 'EC_slab2', 'WC_slab1', 'WC_slab2', 't_slab1', 't_slab2']
indoor_cols = ['Tair', 'Rhair', 'CO2air', 'HumDef', 'PipeLow', 'VentLee', 'Ventwind', 'Tot_PAR', 'Tot_PAR_Lamps', 'EC_drain_PC']
weather_cols = ['Tout', 'Rhout', 'Iglob', 'PARout', 'Pyrgeo', 'Rain', 'Winddir', 'Windsp']
crop_cols = ['Stem_elong', 'Stem_thick', 'Cum_trusses', 'stem_dens', 'plant_dens']
prod_cols = ['ProdA', 'ProdB', 'Weight_fruits_ClassA', 'avg_nr_harvested_trusses']
target_cols = ['Flavour', 'TSS', 'Acid', '%Juice', 'Bite', 'WeightDMC_fruit']

In [7]:
def reshape_sliding(df, cols, steps, stride=1):
    df = df.copy()
    df[cols] = df[cols].astype(np.float32)
    arr = df[cols].values
    if len(arr) < steps:
        return np.empty((0, steps, len(cols)))
    windows = [arr[i:i + steps] for i in range(0, len(arr) - steps + 1, stride)]
    return np.stack(windows)

In [8]:
def compute_delta(mask):
    B, T, D = mask.shape
    delta = np.zeros((B, T, D), dtype=np.float32)
    for b in range(B):
        for d in range(D):
            last_obs = 0
            for t in range(T):
                if mask[b, t, d] == 1:
                    delta[b, t, d] = 0
                    last_obs = 0
                else:
                    last_obs += 1
                    delta[b, t, d] = last_obs
    return delta

In [9]:
# Start with GrodanSens as the high-res base
base = dfs['GrodanSens'].copy()
base = base.dropna(subset=['Date']).sort_values('Date').reset_index(drop=True)
# Define merge partners and their tolerances (1 day is safe for most)
merge_partners = ['GreenhouseClimate', 'Weather', 'Production', 'CropParameters']
for name in merge_partners:
    df = dfs[name].copy()
    # Clean and sort
    df = df.dropna(subset=['Date']).sort_values('Date')
    if '%time' in df.columns:
        df.drop(columns=['%time'], inplace=True)
    try:
        base = pd.merge_asof(
            base, df,
            on='Date',
            direction='nearest',
            tolerance=pd.Timedelta('1D')
        )
    except ValueError as e:
        print(f"[ERROR] Skipped {name} during merge: {e}")
        continue

In [10]:
# Merge TomQuality separately with wider tolerance (2 days)
df_tom = dfs['TomQuality'].copy()
df_tom = df_tom.dropna(subset=['Date']).sort_values('Date')
if '%time' in df_tom.columns:
    df_tom.drop(columns=['%time'], inplace=True)
try:
    base = pd.merge_asof(
        base, df_tom,
        on='Date',
        direction='nearest',
        tolerance=pd.Timedelta('2D')
    )
except ValueError as e:
    print(f"[ERROR] Skipped TomQuality during merge: {e}")

In [11]:
base = base.dropna(subset=['Date']).reset_index(drop=True)

In [12]:
soil_data = reshape_sliding(base, soil_cols, steps=20, stride=1)
soil_mask = (~np.isnan(soil_data)).astype(np.float32)
soil_delta = compute_delta(soil_mask)
soil_data = np.nan_to_num(soil_data)
indoor_data = reshape_sliding(base, indoor_cols, steps=20, stride=1)
weather_data = reshape_sliding(base, weather_cols, steps=10, stride=1)
prod_data = reshape_sliding(base, prod_cols, steps=10, stride=1)
crop_data = reshape_sliding(base, crop_cols, steps=1, stride=1).squeeze(1)

In [13]:
# Full list of possible end dates
window_end_dates = base['Date'].iloc[19:].reset_index(drop=True)
targets = []
valid_indices = []
# Loop only over the number of available soil windows
for i in range(len(soil_data)):
    end_date = window_end_dates[i]
    match = dfs['TomQuality'][(dfs['TomQuality']['Date'] - end_date).abs() <= pd.Timedelta('2D')]
    if not match.empty:
        targets.append(match[target_cols].values[0].astype(np.float32))
        valid_indices.append(i)

In [14]:
if valid_indices:
    soil_data = soil_data[valid_indices]
    soil_mask = soil_mask[valid_indices]
    soil_delta = soil_delta[valid_indices]
    indoor_data = indoor_data[valid_indices]
    weather_data = weather_data[valid_indices]
    prod_data = prod_data[valid_indices]
    crop_data = crop_data[valid_indices]
    targets = torch.tensor(np.array(targets), dtype=torch.float32)
    dataset = TensorDataset(
        torch.tensor(soil_data, dtype=torch.float32),
        torch.tensor(soil_mask, dtype=torch.float32),
        torch.tensor(soil_delta, dtype=torch.float32),
        torch.tensor(indoor_data, dtype=torch.float32),
        torch.tensor(weather_data, dtype=torch.float32),
        torch.tensor(crop_data, dtype=torch.float32),
        torch.tensor(prod_data, dtype=torch.float32),
        targets
)
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
    print(f"Dataloader ready: {len(dataset)} labeled windows matched with TomQuality")
else:
    print("No labeled windows found.")

Dataloader ready: 8936 labeled windows matched with TomQuality


In [15]:
sample_batch = next(iter(dataloader))
soil_x, soil_mask, soil_delta, indoor_x, weather_x, crop_x, prod_x, targets = sample_batch

In [16]:
# Check shapes
print(f"Soil shape: {soil_x.shape}")
print(f"Soil mask shape: {soil_mask.shape}")
print(f"Soil delta shape: {soil_delta.shape}")
print(f"Indoor shape: {indoor_x.shape}")
print(f"Weather shape: {weather_x.shape}")
print(f"Crop shape: {crop_x.shape}")
print(f"Production shape: {prod_x.shape}")
print(f"Targets shape: {targets.shape}")

Soil shape: torch.Size([8, 20, 6])
Soil mask shape: torch.Size([8, 20, 6])
Soil delta shape: torch.Size([8, 20, 6])
Indoor shape: torch.Size([8, 20, 10])
Weather shape: torch.Size([8, 10, 8])
Crop shape: torch.Size([8, 5])
Production shape: torch.Size([8, 10, 4])
Targets shape: torch.Size([8, 6])


In [17]:
# Lengths
total_len = len(dataset)
train_len = int(0.8 * total_len)
val_len = int(0.1 * total_len)
test_len = total_len - train_len - val_len  # handle rounding
# Split dataset
train, val, test = random_split(dataset, [train_len, val_len, test_len])
# DataLoaders
train_loader = DataLoader(train, batch_size=8, shuffle=True)
val_loader = DataLoader(val, batch_size=8, shuffle=False)
test_loader = DataLoader(test, batch_size=8, shuffle=False)
print(f"Split sizes — Train: {len(train)}, Val: {len(val)}, Test: {len(test)}")

Split sizes — Train: 7148, Val: 893, Test: 895


# Model

In [18]:
class GRUD(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=None, device="cpu"):
        super(GRUD, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.device = device
        self.gamma_x = nn.Parameter(torch.Tensor(input_size))
        self.gamma_h = nn.Parameter(torch.Tensor(hidden_size))
        self.z_gate = nn.Linear(input_size * 3 + hidden_size, hidden_size)
        self.r_gate = nn.Linear(input_size * 3 + hidden_size, hidden_size)
        self.h_tilde = nn.Linear(input_size * 3 + hidden_size, hidden_size)
        self.output = nn.Sequential(
            nn.LayerNorm(hidden_size),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, output_size) if output_size else nn.Identity()
        )
        self.reset_parameters()
    def reset_parameters(self):
        nn.init.constant_(self.gamma_x, 0.1)
        nn.init.constant_(self.gamma_h, 0.1)
        for name, param in self.named_parameters():
            if 'weight' in name and param.dim() > 1:
                nn.init.xavier_uniform_(param)
    def forward(self, x, x_mask, x_delta, x_mean=None):
        batch_size, seq_len, _ = x.size()
        if x_mean is None:
            x_mean = torch.mean(x, dim=1, keepdim=True).detach()
        h = torch.zeros(batch_size, self.hidden_size, device=self.device)
        outputs = []
        gamma_h = torch.exp(-F.relu(self.gamma_h)).unsqueeze(0).expand(batch_size, -1)
        for t in range(seq_len):
            x_t = x[:, t, :]
            m_t = x_mask[:, t, :]
            d_t = x_delta[:, t, :]
            gamma_x = torch.exp(-F.relu(self.gamma_x) * d_t)
            x_t_hat = m_t * x_t + (1 - m_t) * (gamma_x * x_t + (1 - gamma_x) * x_mean.squeeze(1))
            h = gamma_h * h
            inputs = torch.cat([x_t_hat, m_t, d_t, h], dim=1)
            z = torch.sigmoid(self.z_gate(inputs))
            r = torch.sigmoid(self.r_gate(inputs))
            h_tilde = torch.tanh(self.h_tilde(torch.cat([x_t_hat, m_t, d_t, r * h], dim=1)))
            h = (1 - z) * h + z * h_tilde
            outputs.append(h.unsqueeze(1))
        outputs = torch.cat(outputs, dim=1)
        return self.output(outputs[:, -1, :])

In [19]:
class MLPBlock(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.mlp(x)

In [20]:
class CNN1D(nn.Module):
    def __init__(self, input_channels, out_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_channels, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Linear(32, out_dim)
        )
    def forward(self, x):
        #[B, C, T]
        return self.net(x)

In [21]:
class GRUEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, h_n = self.gru(x)
        return self.linear(h_n.squeeze(0))

In [22]:
class AttentionFusion(nn.Module):
    def __init__(self, input_dims, fused_dim):
        """
        input_dims: list of dims from each modality (e.g., [64, 64, 32, 64, 32])
        fused_dim: output dimension after fusion
        """
        super().__init__()
        self.target_dim = max(input_dims)
        self.proj = nn.ModuleList([nn.Linear(d, self.target_dim) for d in input_dims])
        self.attn = nn.MultiheadAttention(embed_dim=self.target_dim, num_heads=1, batch_first=True)
        self.norm = nn.LayerNorm(self.target_dim)
        self.dropout = nn.Dropout(0.2)
        self.project = nn.Linear(self.target_dim, fused_dim)
    def forward(self, embeddings):
        # Project all embeddings to common dimension
        projected = [proj(x) for proj, x in zip(self.proj, embeddings)]  # [B, D_i] → [B, target_dim]
        x = torch.stack(projected, dim=1)  # [B, num_modalities, target_dim]
        attn_output, _ = self.attn(x, x, x)
        attn_output = self.norm(attn_output + x)
        pooled = attn_output.mean(dim=1)
        return self.project(self.dropout(pooled))

In [23]:
class PredictionHead(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.head = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),  # Improves stability during training
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, output_dim)  # Matches config['num_targets']
        )
    def forward(self, x):
        return self.head(x)

In [34]:
class HybridAgriModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.grud = GRUD(config['soil_in'], config['soil_hidden'], config['soil_out'], config['device'])
        self.indoor_encoder = GRUEncoder(config['indoor_in'], config['indoor_hidden'], config['indoor_out'])  # Placeholder for TimesNet
        self.weather_encoder = CNN1D(config['weather_in'], config['weather_out'])
        self.crop_encoder = MLPBlock(config['crop_in'], 64, config['crop_out'])
        self.prod_encoder = GRUEncoder(config['prod_in'], config['prod_hidden'], config['prod_out'])
        self.fusion = AttentionFusion([
            config['soil_out'], config['indoor_out'], config['weather_out'], config['crop_out'], config['prod_out']
        ], config['fused_dim'])
        self.head = PredictionHead(config['fused_dim'], config['num_targets'])

    def forward(self, soil_x, soil_mask, soil_delta, indoor_x, weather_x, crop_x, prod_x):
        h_soil = self.grud(soil_x, soil_mask, soil_delta)  # Last time step
        h_indoor = self.indoor_encoder(indoor_x)
        h_weather = self.weather_encoder(weather_x.transpose(1, 2))
        h_crop = self.crop_encoder(crop_x)
        h_prod = self.prod_encoder(prod_x)
        fused = self.fusion([h_soil, h_indoor, h_weather, h_crop, h_prod])
        return self.head(fused)

In [25]:
config_virtual = {
    'soil_in': 6, 'soil_hidden': 64, 'soil_out': 64,
    'indoor_in': 10, 'indoor_hidden': 64, 'indoor_out': 64,
    'weather_in': 8, 'weather_out': 64,
    'crop_in': 5, 'crop_out': 64,
    'prod_in': 4, 'prod_hidden': 64, 'prod_out': 64,
    'fused_dim': 128, 'num_targets': 6,
    'device': 'cpu'
}

# === Dummy inputs that reflect your DataLoader ===
B = 8
sample_inputs = (
    torch.rand(B, 20, config_virtual['soil_in']),       # soil_x
    torch.ones(B, 20, config_virtual['soil_in']),       # soil_mask
    torch.zeros(B, 20, config_virtual['soil_in']),      # soil_delta
    torch.rand(B, 20, config_virtual['indoor_in']),     # indoor_x
    torch.rand(B, config_virtual['weather_in'], 10),    # weather_x
    torch.rand(B, config_virtual['crop_in']),           # crop_x
    torch.rand(B, 10, config_virtual['prod_in'])        # prod_x
)

In [26]:
def validate_model_structure(model, sample_inputs):
    print("Model Architecture Validation:\n")
    logs = []

    def safe_shape(x):
        if isinstance(x, torch.Tensor):
            return list(x.shape)
        elif isinstance(x, (list, tuple)) and isinstance(x[0], torch.Tensor):
            return [list(t.shape) for t in x]
        return str(type(x))

    def hook_fn(module, input, output):
        logs.append({
            "layer": module.__class__.__name__,
            "input_shape": safe_shape(input),
            "output_shape": safe_shape(output)
        })

    hooks = [m.register_forward_hook(hook_fn) for m in model.modules()
             if not isinstance(m, (nn.Sequential, nn.ModuleList)) and m != model]

    model.eval()
    with torch.no_grad():
        model(*sample_inputs)

    for i, log in enumerate(logs):
        print(f"{i:02d} - {log['layer']:20} | Input: {log['input_shape']} -> Output: {log['output_shape']}")
    for h in hooks:
        h.remove()

In [27]:
model = HybridAgriModel(config_virtual)
validate_model_structure(model, sample_inputs)

Model Architecture Validation:

00 - Linear               | Input: [[8, 82]] -> Output: [8, 64]
01 - Linear               | Input: [[8, 82]] -> Output: [8, 64]
02 - Linear               | Input: [[8, 82]] -> Output: [8, 64]
03 - Linear               | Input: [[8, 82]] -> Output: [8, 64]
04 - Linear               | Input: [[8, 82]] -> Output: [8, 64]
05 - Linear               | Input: [[8, 82]] -> Output: [8, 64]
06 - Linear               | Input: [[8, 82]] -> Output: [8, 64]
07 - Linear               | Input: [[8, 82]] -> Output: [8, 64]
08 - Linear               | Input: [[8, 82]] -> Output: [8, 64]
09 - Linear               | Input: [[8, 82]] -> Output: [8, 64]
10 - Linear               | Input: [[8, 82]] -> Output: [8, 64]
11 - Linear               | Input: [[8, 82]] -> Output: [8, 64]
12 - Linear               | Input: [[8, 82]] -> Output: [8, 64]
13 - Linear               | Input: [[8, 82]] -> Output: [8, 64]
14 - Linear               | Input: [[8, 82]] -> Output: [8, 64]
15 - Lin

# Train this bitchass

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

config_real = {
    'soil_in': 6, 'soil_hidden': 64, 'soil_out': 64,
    'indoor_in': 10, 'indoor_hidden': 64, 'indoor_out': 64,
    'weather_in': 8, 'weather_out': 32,
    'crop_in': 5, 'crop_out': 32,
    'prod_in': 4, 'prod_hidden': 64, 'prod_out': 64,
    'fused_dim': 128, 'num_targets': 6,
    'device': device
}

Using device: cuda


In [36]:
model = HybridAgriModel(config_real).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.MSELoss()
epochs = 100

In [37]:
def train_one_epoch(loader):
    model.train()
    total_loss = 0
    for batch in tqdm(loader, desc="Train"):
        soil_x, soil_mask, soil_delta, indoor_x, weather_x, crop_x, prod_x, y = [b.to(device) for b in batch]
        optimizer.zero_grad()
        out = model(soil_x, soil_mask, soil_delta, indoor_x, weather_x, crop_x, prod_x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [38]:
def validate(loader):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(loader, desc="Val"):
            soil_x, soil_mask, soil_delta, indoor_x, weather_x, crop_x, prod_x, y = [b.to(device) for b in batch]
            out = model(soil_x, soil_mask, soil_delta, indoor_x, weather_x, crop_x, prod_x)
            loss = criterion(out, y)
            val_loss += loss.item()
    return val_loss / len(loader)

In [39]:
def evaluate(loader):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for batch in tqdm(loader, desc="Test"):
            soil_x, soil_mask, soil_delta, indoor_x, weather_x, crop_x, prod_x, y = [b.to(device) for b in batch]
            out = model(soil_x, soil_mask, soil_delta, indoor_x, weather_x, crop_x, prod_x)
            preds.append(out.cpu().numpy())
            targets.append(y.cpu().numpy())
    preds = np.concatenate(preds)
    targets = np.concatenate(targets)
    mse = mean_squared_error(targets, preds)
    mae = mean_absolute_error(targets, preds)
    print(f"\nTest MSE: {mse:.4f}, MAE: {mae:.4f}")

In [40]:
for epoch in range(1, epochs + 1):
    train_loss = train_one_epoch(train_loader)
    val_loss = validate(val_loader)
    print(f"[Epoch {epoch:03}] Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

Val: 100%|██████████| 112/112 [00:01<00:00, 69.27it/s]


[Epoch 001] Train Loss: nan | Val Loss: nan


Val: 100%|██████████| 112/112 [00:01<00:00, 69.12it/s]


[Epoch 002] Train Loss: nan | Val Loss: nan


Val: 100%|██████████| 112/112 [00:01<00:00, 69.38it/s]


...

[Epoch 006] Train Loss: nan | Val Loss: nan


Val: 100%|██████████| 112/112 [00:01<00:00, 64.41it/s]


[Epoch 007] Train Loss: nan | Val Loss: nan


Train:  70%|███████   | 627/894 [00:26<00:11, 23.36it/s]


KeyboardInterrupt: 

In [ ]:
evaluate(test_loader)